### 2. Multiple Linear Regression
### 원-핫 인코딩


In [10]:
import pandas as pd

In [11]:
dataset = pd.read_csv('./MultipleLinearRegressionData.csv')
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values
X, y

(array([[0.5, 3, 'Home'],
        [1.2, 4, 'Library'],
        [1.8, 2, 'Cafe'],
        [2.4, 0, 'Cafe'],
        [2.6, 2, 'Home'],
        [3.2, 0, 'Home'],
        [3.9, 0, 'Library'],
        [4.4, 0, 'Library'],
        [4.5, 5, 'Home'],
        [5.0, 1, 'Cafe'],
        [5.3, 2, 'Cafe'],
        [5.8, 0, 'Cafe'],
        [6.0, 3, 'Library'],
        [6.1, 1, 'Cafe'],
        [6.2, 1, 'Library'],
        [6.9, 4, 'Home'],
        [7.2, 2, 'Cafe'],
        [8.4, 1, 'Home'],
        [8.6, 1, 'Library'],
        [10.0, 0, 'Library']], dtype=object),
 array([ 10,   8,  14,  26,  22,  30,  42,  48,  38,  58,  60,  72,  62,
         68,  72,  58,  76,  86,  90, 100]))

In [12]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# "encoder" : 원-핫 인코더 적용
# drop='first' : 다중공산성 문제를 없애기 위하여 적용 (n-1)
# [2] : Index 2에 해당하는 column index만 적용
# remainder : 원-핫 인코딩을 적용하지 않는 나머지 columns들에 대해서 어떻게 적용할 것인지 (=> passthrough : 그대로 둠)
ct = ColumnTransformer(transformers=[("encoder", OneHotEncoder(drop='first'), [2])], remainder="passthrough")
X = ct.fit_transform(X) # 원-핫 인코딩을 통해서 0, 1 로 바뀌고 X에 적용해줌
X

# 1 0 : Home
# 0 1 : Library
# 0 0 : Cafe
# 두 개의 컬럼 추가 & 기존의 place column은 제거됨.
# => 숫자가 아닌 범주형 데이터에 대해서 처리하는 방법을 실습

array([[1.0, 0.0, 0.5, 3],
       [0.0, 1.0, 1.2, 4],
       [0.0, 0.0, 1.8, 2],
       [0.0, 0.0, 2.4, 0],
       [1.0, 0.0, 2.6, 2],
       [1.0, 0.0, 3.2, 0],
       [0.0, 1.0, 3.9, 0],
       [0.0, 1.0, 4.4, 0],
       [1.0, 0.0, 4.5, 5],
       [0.0, 0.0, 5.0, 1],
       [0.0, 0.0, 5.3, 2],
       [0.0, 0.0, 5.8, 0],
       [0.0, 1.0, 6.0, 3],
       [0.0, 0.0, 6.1, 1],
       [0.0, 1.0, 6.2, 1],
       [1.0, 0.0, 6.9, 4],
       [0.0, 0.0, 7.2, 2],
       [1.0, 0.0, 8.4, 1],
       [0.0, 1.0, 8.6, 1],
       [0.0, 1.0, 10.0, 0]], dtype=object)

### 데이터 세트 분리

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

### 학습 (다중 선형 회귀)

In [15]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
reg.fit(X_train, y_train) # 학습(모델 생성)

LinearRegression()

### 예측 값과 실제 값 비교 (테스트 세트)

In [16]:
y_pred = reg.predict(X_test)
y_pred

array([ 92.15457859,  10.23753043, 108.36245302,  38.14675204])

In [17]:
y_test

array([ 90,   8, 100,  38])

In [18]:
reg.coef_
# 집, 카페, 점수, 결석

array([-5.82712824, -1.04450647, 10.40419528, -1.64200104])

In [19]:
reg.intercept_

5.365006706544776

### 모델 평가

In [20]:
reg.score(X_train, y_train) # 훈련 세트

0.9623352565265527

In [21]:
reg.score(X_test, y_test) # 테스트 세트

0.9859956178877446

### 다양한 평가 지표 (회귀 모델)

1. MAE (Mean Absolute Error) : (실제 값과 예측 값) 차이의 절대값
2. MSE (Mean Squared Error) : 차이의 제곱
3. RMSE (Root Mean Squared Error) : 차이의 제곱에 루트
4. R2 : 결정 계수

> R2는 1에 가까울수록, 나머지는 0에 가까울수록 데이터를 잘 표현한다는 의미이다.

In [24]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test, y_pred) # MAE : |실제값(y_test) - 예측값(y_pred)|

3.2253285188287975

In [25]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, y_pred) # MSE : (실제값(y_test) - 예측값(y_pred))^2

19.900226981514905

In [26]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, y_pred, squared=False) # RMSE : (실제값(y_test) - 예측값(y_pred))^2

4.4609670455535655

In [27]:
from sklearn.metrics import r2_score
r2_score(y_test, y_pred) # R2

# 1에 가까울수록 좋은 모델
# 위의 reg의 모델 평가를 했을 때와 동일한 값이 나온다
# => 즉, 기본적으로 reg의 모델 평가는 r2_score를 이용하지만 다른 모델로 평가를 해보고 싶을 때 MAE, MSE, RMSE 를 이용

0.9859956178877446